# Calculating recurrence scores in time series

In [2]:
using CABLAB
using ImageMagick #for inline Map plots

We define the path to the datacube, choose some variables and read a geographical region into memory.

In [3]:
c             = Cube("/Net/Groups/BGI/scratch/DataCube/v1/brockmann-consult.de/datacube")
vars          = ["Emission","SoilMoisture","t2m"];
cdata         = getCubeData(c,latitude=(35,65), longitude=(-10,35),variable=vars);

Here starts the actual processing step. Note that each function call has the result of the previous call as its argument. The following processing steps are applied:

- gap Filling with the mean seasonal cycle
- calculating anomalies, i.e. subtract the mean seasonal cycle
- normalize the different variables to unit variance
- caculate recurrence scores to detect outliers

All these processing steps are implemented in the DAT.

In [4]:
@time cube_filled     = mapCube(gapFillMSC,cdata);

 16.767042 seconds (16.92 M allocations: 1007.194 MB, 2.23% gc time)


In [5]:
@time cube_anomalies  = mapCube(removeMSC,cube_filled);

  5.193918 seconds (6.00 M allocations: 263.949 MB, 4.11% gc time)


In [6]:
@time cube_normalized = mapCube(normalize,cube_anomalies);

  3.595263 seconds (3.14 M allocations: 171.790 MB, 1.32% gc time)


In [7]:
@time scores          = mapCube(recurrences,cube_normalized,0.5,5);

 51.831484 seconds (47.93 M allocations: 2.023 GB, 0.95% gc time)


Calculating the scores takes a bit long, so we can repeat this after adding a few more processors.

In [8]:
addprocs(5);

In [10]:
CABLAB.DAT.init_DATworkers()

In [ ]:
@time scores          = mapCube(recurrences,cube_normalized,5.0,5);
rmprocs(workers());

Here follow some interactive time series plots of the different variables, anomalies, and the resulting scores. 3 Extreme events would be detected here. 

In [ ]:
plotTS(cdata)

In [ ]:
plotTS(cube_anomalies)

In [ ]:
plotTS(cube_normalized)

In [ ]:
plotTS(scores)

In [ ]:
plotMAP(scores,dmin=200.f0,dmax=500.f0)